In [1]:
!ln -s /mnt/data/annotations/by_release/hg19/hg19.chrom.sizes .

ln: failed to create symbolic link './hg19.chrom.sizes': File exists


In [2]:
%%bash
for idx in 0 1 2 3 4 5; do
    bedfile="../coordinates/coordinates_"$idx".bed"
    cat $bedfile | perl -lane 'print $F[0]."\t".($F[1]+$F[9]-500)."\t".($F[1]+$F[9]+500)' > expanded_coordinates_$idx.bed
    bedtools getfasta -fi /mnt/data/annotations/by_release/hg19/male.hg19.fa -bed expanded_coordinates_$idx.bed > expanded_coordinates_$idx.fa
done

In [3]:
indices = [0,1,2,3,4,5]
fastafiles = ["expanded_coordinates_"+str(idx)+".fa" for idx in indices]
model_hdf5 = "/srv/scratch/annashch/deeplearning/encode4crispr/k562_dnase/regression/DNASE.K562.regressionlabels.allbins.0"

In [4]:
import numpy as np

#this is set up for 1d convolutions where examples
#have dimensions (len, num_channels)
#the channel axis is the axis for one-hot encoding.
def one_hot_encode_along_channel_axis(sequence):
    to_return = np.zeros((len(sequence),4), dtype=np.int8)
    seq_to_one_hot_fill_in_array(zeros_array=to_return,
                                 sequence=sequence, one_hot_axis=1)
    return to_return

def seq_to_one_hot_fill_in_array(zeros_array, sequence, one_hot_axis):
    assert one_hot_axis==0 or one_hot_axis==1
    if (one_hot_axis==0):
        assert zeros_array.shape[1] == len(sequence)
    elif (one_hot_axis==1): 
        assert zeros_array.shape[0] == len(sequence)
    #will mutate zeros_array
    for (i,char) in enumerate(sequence):
        if (char=="A" or char=="a"):
            char_idx = 0
        elif (char=="C" or char=="c"):
            char_idx = 1
        elif (char=="G" or char=="g"):
            char_idx = 2
        elif (char=="T" or char=="t"):
            char_idx = 3
        elif (char=="N" or char=="n"):
            continue #leave that pos as all 0's
        else:
            raise RuntimeError("Unsupported character: "+str(char))
        if (one_hot_axis==0):
            zeros_array[char_idx,i] = 1
        elif (one_hot_axis==1):
            zeros_array[i,char_idx] = 1

sequences_sets = [[x[1].rstrip() for x in enumerate(open(fastafile)) if x[0]%2==1]
                  for fastafile in fastafiles]

In [11]:
import deeplift
from deeplift.conversion import kerasapi_conversion as kc
from deeplift.util import get_shuffle_seq_ref_function
from deeplift.dinuc_shuffle import dinuc_shuffle

deeplift_model =\
    kc.convert_model_from_saved_files(
        model_hdf5,
        nonlinear_mxts_mode=deeplift.layers.NonlinearMxtsMode.Rescale)
deeplift_contribs_func = deeplift_model.get_target_contribs_func(
                            find_scores_layer_idx=0,
                            target_layer_idx=-1)
onehot_func = lambda x: np.array([one_hot_encode_along_channel_axis(seq)[None,:,:] for seq in x])
scoring_func = get_shuffle_seq_ref_function(
    #score_computation_function is the original function to compute scores
    score_computation_function=deeplift_contribs_func,
    #shuffle_func is the function that shuffles the sequence
    #technically, given the background of this simulation, randomly_shuffle_seq
    #makes more sense. However, on real data, a dinuc shuffle is advisable due to
    #the strong bias against CG dinucleotides
    shuffle_func=dinuc_shuffle,
    one_hot_func=None)

nonlinear_mxts_mode is set to: Rescale
Heads-up: current implementation assumes maxpool layer is followed by a linear transformation (conv/dense layer)
Heads-up: current implementation assumes maxpool layer is followed by a linear transformation (conv/dense layer)
Heads-up: current implementation assumes maxpool layer is followed by a linear transformation (conv/dense layer)


In [ ]:
for idx,sequences in enumerate(sequences_sets):
    onehot = onehot_func(sequences)
    deeplift_scores = scoring_func(
        task_idx=0,
        input_data_sequences=onehot,
        num_refs_per_seq=10,
        batch_size=200,
        progress_update=1000)
    print(deeplift_scores.shape)
    onehot = np.squeeze(onehot)
    deeplift_scores = np.squeeze(deeplift_scores)
    projected_deeplift = np.sum(deeplift_scores, axis=-1)[:,:,None]*onehot
    print(projected_deeplift.shape)
    np.save("deeplift_scores_"+str(idx)+".npy", projected_deeplift)

1000 reference seqs generated
2000 reference seqs generated
3000 reference seqs generated
4000 reference seqs generated
5000 reference seqs generated
6000 reference seqs generated
7000 reference seqs generated
8000 reference seqs generated
9000 reference seqs generated
10000 reference seqs generated
11000 reference seqs generated
12000 reference seqs generated
13000 reference seqs generated
14000 reference seqs generated
15000 reference seqs generated
16000 reference seqs generated
